### Création du dataset pour le modèle NER (CamemBERT)

Création du dataset pour le deuxième modèle qui détermine la ville d'arrivée et la ville de départ.

In [1]:
import pandas as pd
import random
import string

#### Extraire les villes depuis un fichier CSV

In [2]:
cities_df = pd.read_csv('communes-departement-region.csv')
all_cities = cities_df['nom_commune_complet'].tolist()
print(all_cities[:10])

["L'Abergement-Clémenciat", "L'Abergement-de-Varey", 'Ambérieu-en-Bugey', 'Ambérieux-en-Dombes', 'Ambléon', 'Ambronay', 'Ambutrix', 'Andert-et-Condon', 'Anglefort', 'Apremont']


#### Créer les phrases à partir de templates

In [3]:
cities = [
    "Paris", "Lyon", "Marseille", "Toulouse", "Nice", "Nantes", "Strasbourg", "Montpellier", "Lille",
    "Rennes", "Reims", "lyon", "Saint-Étienne", "Toulon", "Grenoble", "Dijon", "Angers", "Nîmes", "Villeurbanne", "angers",
    "Cannes", "Colmar", "Béziers", "Valence", "Quimper", "Chambéry", "châteauroux", "Annecy", "Bayonne", "Tarbes", "Albi",
    "Bourges", "Niort", "Châteauroux", "colmar", "Carcassonne", "Cherbourg", "Mâcon", "Vannes", "Lorient",
    "Clermont-Ferrand", "Le Havre", "Le Mans", "Aix-en-Provence", "Brest", "Limoges", "Bordeaux", "Tours", "Amiens", "Perpignan", "Metz", "Besançon",
    "Orléans", "Mulhouse", "Rouen", "Caen", "Nancy", "Saint-Denis", "Argenteuil", "Montreuil", "Roubaix", "Dunkerque",
    "Tourcoing", "Nanterre", "Avignon", "Créteil", "Poitiers", "Versailles", "Pau", "Courbevoie", "La Rochelle", "Antibes",
    "Ajaccio", "Bastia", "Calais", "Évreux", "Périgueux", "Saint-Brieuc", "Brive-la-Gaillarde", "Angoulême", "La Roche-sur-Yon", "Cholet", "Saint-Nazaire",
    "Sète", "Narbonne", "Montauban", "Alès", "Martigues", "Gap", "Salon-de-Provence", "Arles", "Istres", "Fréjus", "Draguignan",
    "Manosque", "Digne-les-Bains", "lyon", "Briançon", "Sisteron", "Forcalquier", "Grasse", "Cagnes-sur-Mer", "Menton", "Saint-Tropez", "Hyères"
]
    
valid_sentences_template = [
    "Je veux voyager de {start} à {end} .",
    "Je voudrais un billet de {start} à {end} .",
    "Comment puis-je aller de {start} à {end} ?",
    "Pouvez-vous me trouver un train de {start} à {end} ?",
    "Je souhaite réserver un billet de {start} à {end}.",
    "Y a-t-il des trains de {start} à {end} demain ?",
    "Je dois aller de {start} à {end} .",
    "Quel est le prochain train de {start} à {end} ?",
    "Je cherche un billet de {start} à {end} .",
    "Comment puis-je me rendre de {start} à {end} ?",
    "Je veux aller de {start} à {end} .",
    "Pouvez-vous me trouver un train de {start} à {end} ?",
    "Je souhaite réserver un billet de {start} à {end}.",
    "Y a-t-il des trains de {start} à {end} demain ?",
    "Je dois aller de {start} à {end} .",
    "Quel est le prochain train de {start} à {end} ?",
    "Je cherche un billet de {start} à {end} .",
    "Comment puis-je me rendre à {end} depuis {start} ?",
    "Je veux aller à {end} depuis {start} .",
    "Pouvez-vous me trouver un train de {start} à {end} ?",
    "Je veux réserver un voyage de {start} à {end} .",
    "Quel est le prix d'un billet de {start} à {end} ?",
    "Y a-t-il un train direct de {start} à {end} ?",
    "Combien de temps faut-il pour aller de {start} à {end} ?",
    "Quels sont les horaires des trains de {start} à {end} ?",
    "Je voudrais connaître les options de voyage de {start} à {end} .",
    "Pouvez-vous m'aider à trouver un train de {start} à {end} ?",
    "Je cherche des informations sur les trains de {start} à {end} .",
    "Quels sont les arrêts entre {start} et {end} ?",
    "Y a-t-il des réductions pour un billet de {start} à {end} ?",
    "Je veux savoir s'il y a des trains de {start} à {end} ce soir .",
    "Pouvez-vous vérifier les disponibilités pour un train de {start} à {end} ?",
    "Je veux réserver un billet aller-retour de {start} à {end} .",
    "Quels sont les tarifs pour un voyage de {start} à {end} ?",
    "Y a-t-il des trains de nuit de {start} à {end} ?",
    "Je veux connaître les horaires des trains de {start} à {end} demain .",
    "Pouvez-vous me donner des informations sur les trains de {start} à {end} ?",
    "Je cherche un train de {start} à {end} avec une correspondance .",
    "Quels sont les services disponibles à bord des trains de {start} à {end} ?",
    "Je veux réserver un billet de {start} à {end} pour ce week-end .",
    "Comment puis-je voyager de {start} à {end} en toute sécurité ?",
    "Je recherche un moyen de transport de {start} à {end}.",
    "Y a-t-il des options de voyage de {start} à {end} en bus ?",
    "Quelle est la meilleure façon d'aller de {start} à {end} rapidement ?",
    "Pouvez-vous recommander un itinéraire de {start} à {end} ?",
    "Je souhaite réserver un vol de {start} à {end}.",
    "Y a-t-il des vols directs de {start} à {end} ?",
    "Je m'intéresse aux voyages de {start} à {end}.",
    "Quel est le moyen de transport le plus économique de {start} à {end} ?",
    "Je cherche des conseils pour voyager de {start} à {end}.",
    "Comment puis-je me rendre à {end} en partant de {start} ?",
    "Je veux aller de {start} à {end} en voiture.",
    "Pouvez-vous me recommander un trajet de {start} à {end} en métro ?",
    "Je souhaite réserver un taxi de {start} à {end}.",
    "Y a-t-il des bus directs de {start} à {end} ?",
    "Je dois me déplacer de {start} à {end} rapidement.",
    "Quel est le prochain vol de {start} à {end} ?",
    "Je cherche un billet d'avion de {start} à {end}.",
    "Comment puis-je voyager de {start} à {end} en toute tranquillité ?",
    "Je prévois un déplacement de {start} à {end}.",
    "Pouvez-vous me recommander un moyen de transport de {start} à {end} ?",
    "Je souhaite réserver un covoiturage de {start} à {end}.",
    "Y a-t-il des vols de {start} à {end} demain ?",
    "Je dois me rendre en {end} depuis {start}.",
    "Quel est le prochain bus de {start} à {end} ?",
    "Je cherche un billet de {start} à {end} en avion.",
    "Comment puis-je me déplacer de {start} à {end} de manière écologique ?",
    "Je veux aller en {end} depuis {start} en bus.",
    "Pouvez-vous me recommander un train rapide de {start} à {end} ?",
    "Je souhaite réserver un trajet en bus de {start} à {end}.",
    "Y a-t-il des trains directs de {start} à {end} demain ?",
    "Je dois me rendre de {start} à {end} en urgence.",
    "Quel est le prochain bus direct de {start} à {end} ?",
    "Je cherche un billet de {start} à {end} en train.",
    "Comment puis-je me rendre à {end} depuis {start} en métro ?",
    "Je veux aller à {end} depuis {start} en tramway.",
    "Pouvez-vous me recommander un bus de {start} à {end} avec WiFi ?",
    "Je souhaite réserver un billet de {start} à {end} en TGV.",
    "Y a-t-il des trains de {start} à {end} ce week-end ?"
]


In [4]:
def generate_questions(templates, num_questions=2000):
    questions = []
    for _ in range(num_questions):
        start = random.choice(all_cities)
        end = random.choice(all_cities)
        if start != end:  # Ensure start and end are not the same
            template = random.choice(templates)
            question = template.format(start=start, end=end)
            questions.append((question, start, end))
    return questions

#### Créer des phrases au bon format

In [5]:
valid_sentences = generate_questions(valid_sentences_template, 4000)

In [6]:
df = pd.DataFrame(valid_sentences, columns=['question', 'start', 'end'])
df['sentence_id'] = df.index

sample_df = df.sample(n=10, random_state=42)
print(sample_df)

                                               question  \
555   Je voudrais un billet de Abeilhan à Saint-Denœ...   
3491  Comment puis-je voyager de Entraunes à Souanya...   
527   Je veux savoir s'il y a des trains de Woimbey ...   
3925  Y a-t-il des réductions pour un billet de Veyr...   
2989  Pouvez-vous me recommander un trajet de Mulcen...   
70    Je prévois un déplacement de Courtrizy-et-Fuss...   
1756  Je voudrais connaître les options de voyage de...   
211   Je veux aller à Salonnes depuis Saint-Clément-...   
196   Comment puis-je me rendre à Héricourt depuis L...   
803   Pouvez-vous me recommander un bus de Sainte-Fo...   

                        start                         end  sentence_id  
555                  Abeilhan                Saint-Denœux          555  
3491                Entraunes                    Souanyas         3491  
527                   Woimbey        Saint-Aubin-des-Bois          527  
3925        Veyrines-de-Vergt  Château-Arnoux-Saint-Auban 

##### Conversion au format NER

In [7]:
# Keep ponctuation
def convert_to_ner_format(row):
    tokens = row['question'].split()
    ner_format = []
    start_tokens = row['start'].split()
    end_tokens = row['end'].split()

    i = 0
    while i < len(tokens):
        token = tokens[i]
        if token in start_tokens:
            for j, start_token in enumerate(start_tokens):
                label = 'B_ville_départ' if j == 0 else 'I_ville_départ'
                ner_format.append((start_token, label, row['sentence_id']))
                i += 1
        elif token in end_tokens:
            for j, end_token in enumerate(end_tokens):
                label = 'B_ville_arrivée' if j == 0 else 'I_ville_arrivée'
                ner_format.append((end_token, label, row['sentence_id']))
                i += 1
        else:
            ner_format.append((token, 'O', row['sentence_id']))
            i += 1

    return ner_format

# Remove ponctuation
# def convert_to_ner_format(row):
#     tokens = row['question'].split()
#     ner_format = []
#     start_tokens = row['start'].split()
#     end_tokens = row['end'].split()

#     i = 0
#     while i < len(tokens):
#         token = tokens[i]
#         token_clean = token.strip(string.punctuation)  # Remove punctuation from the token
#         if token_clean in start_tokens:
#             for j, start_token in enumerate(start_tokens):
#                 label = 'B_ville_départ' if j == 0 else 'I_ville_départ'
#                 ner_format.append((start_token, label, row['sentence_id']))
#                 i += 1
#         elif token_clean in end_tokens:
#             for j, end_token in enumerate(end_tokens):
#                 label = 'B_ville_arrivée' if j == 0 else 'I_ville_arrivée'
#                 ner_format.append((end_token, label, row['sentence_id']))
#                 i += 1
#         else:
#             ner_format.append((token, 'O', row['sentence_id']))
#             i += 1

#     return ner_format

# Sépare la ponctuation
# import re

# def tokenize_with_punctuation(text):
#     # Utilise une expression régulière pour séparer les mots et la ponctuation
#     return re.findall(r'\w+|[^\w\s]', text, re.UNICODE)

# def convert_to_ner_format(row):
#     tokens = tokenize_with_punctuation(row['question'])
#     ner_format = []
#     start_tokens = tokenize_with_punctuation(row['start'])
#     end_tokens = tokenize_with_punctuation(row['end'])

#     i = 0
#     while i < len(tokens):
#         token = tokens[i]
#         if token in start_tokens:
#             for j, start_token in enumerate(start_tokens):
#                 label = 'B_ville_départ' if j == 0 else 'I_ville_départ'
#                 ner_format.append((start_token, label, row['sentence_id']))
#                 i += 1
#         elif token in end_tokens:
#             for j, end_token in enumerate(end_tokens):
#                 label = 'B_ville_arrivée' if j == 0 else 'I_ville_arrivée'
#                 ner_format.append((end_token, label, row['sentence_id']))
#                 i += 1
#         else:
#             ner_format.append((token, 'O', row['sentence_id']))
#             i += 1

#     return ner_format

In [8]:
# Apply the conversion to the entire dataframe
ner_data = df.apply(convert_to_ner_format, axis=1)

# Flatten the list of tuples and create a new DataFrame
flattened_ner_data = [item for sublist in ner_data for item in sublist]
ner_df = pd.DataFrame(flattened_ner_data, columns=['token', 'label', 'sentence_id'])

# Add sentence_id column
# ner_df['sentence_id'] = (ner_df['token'] == '?').cumsum()

# Define the label mapping
label_mapping = {"O": 0, "B_ville_départ": 1, "I_ville_départ": 2, "B_ville_arrivée": 3, "I_ville_arrivée": 4}

# Add ner_tag column with numerical labels
ner_df['ner_tag'] = ner_df['label'].map(label_mapping)

# Save the NER formatted data to a CSV file
ner_df.to_csv('ner_sentences_dataset.csv', index=False)

# Display the first 10 rows of the NER formatted data
print(ner_df.head(50))

                        token            label  sentence_id  ner_tag
0                     Comment                O            0        0
1                     puis-je                O            0        0
2                     voyager                O            0        0
3                          de                O            0        0
4                 Marcheprime   B_ville_départ            0        1
5                           à                O            0        0
6                          La  B_ville_arrivée            0        3
7                Limouzinière  I_ville_arrivée            0        4
8                          en                O            0        0
9                       toute                O            0        0
10                   sécurité                O            0        0
11                          ?                O            0        0
12                    Combien                O            1        0
13                         de     